In [1]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import os

In [ ]:
# Reading Excel File
df = pd.read_excel('./data/input.xlsx')

In [6]:
df.count()

URL_ID    147
URL       147
dtype: int64

In [ ]:
# Creating Articles Directory
output_dir = 'articles'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
url = df.iloc[4]['URL']
url

'https://insights.blackcoffer.com/development-of-ea-robot-for-automated-trading/'

In [ ]:
# Checking if the website is Scrappable 200-OK
response = requests.get('https://insights.blackcoffer.com/securing-sensitive-financial-data-with-privacy-preserving-machine-learning-for-predictive-analytics/')
soup = BeautifulSoup(response.content, 'html.parser')
response


<Response [200]>

In [14]:
title = soup.find('h1', class_='entry-title').get_text(strip=True)
title

'Amazon Buy Bot, an Automation AI tool to Auto-Checkouts'

In [13]:
article_div = soup.find('div', class_='td-post-content tagdiv-type')
article_text = article_div.get_text(separator=' ', strip=True) if article_div else "No Content"
article_text = article_text.replace('_', '')
article_text

'Client Background Client: A leading consulting firm in the USA Industry Type: Consulting Services: Management consultant Organization Size: 100+ Project Objective The main objective of this project is to build the automation tool to buy product on amazon. Project Description This project is basically completed using selenium and Python. All we have done is write a python script for automation using Selenium. Make some clicks use logics to check item is in stock or not. If the item is in stock then it buys the product otherwise repeat the process again. Our Solution A simple python code which uses selenium web driver to do all work. Project Deliverables Python Code Tools used Selenium Webdriver Language/techniques used Python Skills used Web Scraping Selenium Project Snapshots'

In [12]:
# Remove anything that is not a letter, number, space, or punctuation
import re
content_elements = article_div.find_all()  # Get both <h1> and <p> tags
article_text = ' '.join([element.get_text(strip=True) for element in content_elements])

article_text = re.sub(r'[^a-zA-Z0-9\s\.,!?;:()\-]', '', article_text)  # Keep letters, numbers, spaces, and common punctuation
article_text


'Client Background Client Background Client:A leading consulting firm in the USA Client: Industry Type:Consulting Industry Type: Services:Management consultant Services: Organization Size:100 Organization Size: Project Objective Project Objective The main objective of this project is to build the automation tool to buy product on amazon. Project Description Project Description This project is basically completed using selenium and Python. All we have done is write a python script for automation using Selenium. Make some clicks use logics to check item is in stock or not. If the item is in stock then it buys the product otherwise repeat the process again. Our Solution Our Solution A simple python code which uses selenium web driver to do all work. Project Deliverables Project Deliverables Python Code Python Code Tools used Tools used Selenium Webdriver Selenium Webdriver Languagetechniques used Languagetechniques used Python Python Skills used Skills used Web Scraping Selenium Project S

In [15]:
# Extract the article text from the div and removed extra line that was coming in form of "__________________________"
if article_div:
    article_text = article_div.get_text(separator=' ', strip=True)
    # Remove underscores
    article_text = article_text.replace('_', '')
else:
    article_text = "No Content"

print(article_text)


Client Background Client: A leading consulting firm in the USA Industry Type: Consulting Services: Management consultant Organization Size: 100+ Project Objective The main objective of this project is to build the automation tool to buy product on amazon. Project Description This project is basically completed using selenium and Python. All we have done is write a python script for automation using Selenium. Make some clicks use logics to check item is in stock or not. If the item is in stock then it buys the product otherwise repeat the process again. Our Solution A simple python code which uses selenium web driver to do all work. Project Deliverables Python Code Tools used Selenium Webdriver Language/techniques used Python Skills used Web Scraping Selenium Project Snapshots


In [38]:
article_div = soup.find('div', class_='td-post-content tagdiv-type')
article_text = article_div.get_text(separator=' ', strip=True) if article_div else "No Content"
article_text = article_text.replace('_', '')
article_text

content = f"{title}\n\n{article_text}"

# Save to a text file named after URL_ID
file_name = os.path.join('articles', f"{df.iloc[0]['URL_ID']}.txt")
with open(file_name, 'w', encoding='utf-8') as file:
    file.write(content)

print(f"Saved article {df.iloc[0]['URL_ID']} to {file_name}")

Saved article Netclan20241017 to articles\Netclan20241017.txt


In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests.adapters import HTTPAdapter
from requests.exceptions import Timeout, RequestException
from urllib3.util.retry import Retry


session = requests.Session()
retry_strategy = Retry(
    total=3,  
    backoff_factor=1,  # Multiplier for delay between retries: 1s, 2s, 4s, etc.
    status_forcelist=[429, 500, 502, 503, 504], 
    allowed_methods=["GET"] 
)
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("http://", adapter) # Allowing requests from http and https only
session.mount("https://", adapter)

# Process each URL in the DataFrame
for _, row in df.iterrows():
    url = row['URL']
    url_id = row['URL_ID']
    if url:
        try:
            # Send the request with a timeout
            response = session.get(url, timeout=5)
            response.raise_for_status() 

            # Parse the HTML content
            soup = BeautifulSoup(response.content, 'html.parser')

            # Extract the title (if available)
            title_element = soup.find('h1', class_='entry-title')
            title = title_element.get_text(strip=True) if title_element else "No Title"

            # Extract the article content (if available)
            article_div = soup.find('div', class_='td-post-content tagdiv-type')
            article_text = article_div.get_text(separator=' ', strip=True) if article_div else "No Content Available"

            # Remove any underscores from the article content
            article_text = article_text.replace('_', '')

            # Combine title and article content
            content = f"{title}\n\n{article_text}"

            # Save to a text file named after URL_ID
            file_name = os.path.join('articles', f"{url_id}.txt")
            with open(file_name, 'w', encoding='utf-8') as file:
                file.write(content)

            print(f"Saved article {url_id} to {file_name}")

        except Timeout:
            print(f"Timeout occurred for {url} (ID: {url_id}). Skipping.")
        except RequestException as e:
            print(f"Error fetching {url} (ID: {url_id}): {e}")
    else:
        break


Saved article Netclan20241017 to articles\Netclan20241017.txt
Saved article Netclan20241018 to articles\Netclan20241018.txt
Saved article Netclan20241019 to articles\Netclan20241019.txt
Saved article Netclan20241020 to articles\Netclan20241020.txt
Saved article Netclan20241021 to articles\Netclan20241021.txt
Saved article Netclan20241022 to articles\Netclan20241022.txt
Saved article Netclan20241023 to articles\Netclan20241023.txt
Saved article Netclan20241024 to articles\Netclan20241024.txt
Saved article Netclan20241025 to articles\Netclan20241025.txt
Saved article Netclan20241026 to articles\Netclan20241026.txt
Saved article Netclan20241027 to articles\Netclan20241027.txt
Saved article Netclan20241028 to articles\Netclan20241028.txt
Saved article Netclan20241029 to articles\Netclan20241029.txt
Saved article Netclan20241030 to articles\Netclan20241030.txt
Saved article Netclan20241031 to articles\Netclan20241031.txt
Saved article Netclan20241032 to articles\Netclan20241032.txt
Saved ar

In [ ]:
# Extraction Completed